In [1]:
from langchain.tools import tool


@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

In [2]:
@tool("web_search")  # Custom name
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

web_search


In [3]:
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

print(calc.description)

Performs arithmetic calculations. Use this for any math problems.


In [4]:
from pydantic import BaseModel, Field
from typing import Literal


class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [5]:
from langchain.tools import tool, ToolRuntime


# Access the current conversation state
@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

In [6]:
from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime


# Update the conversation history by removing all messages
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""

    return Command(
        update={
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
        }
    )

# Update the user_name in the agent state
@tool
def update_user_name(
    new_name: str,
    runtime: ToolRuntime
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

In [ ]:
import os
from dataclasses import dataclass
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool, ToolRuntime


load_dotenv()

USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
)
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

agent.invoke(
    {"messages": [HumanMessage("What's my current balance?")]},
    context=UserContext(user_id="user123")
)

{'messages': [HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='a854d09e-1a76-449b-99a6-646985482c20'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 252, 'total_tokens': 264, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-34219d76-05e1-4d19-8209-30db90c3a44a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--df0cbb46-f5e2-44a5-a0ef-126f74ad5209-0', tool_calls=[{'name': 'get_account_info', 'args': {}, 'id': 'call_f4c8a185f26748c5828c6785', 'type': 'tool_call'}], usage_metadata={'input_tokens': 252, 'output_tokens': 12, 'total_tokens': 264, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  ToolMessage(content='Account holder: Alice Johnson\nType: Premium\nBa

In [8]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# Access memory
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

store = InMemoryStore()
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

In [9]:
# First session: save user info
agent.invoke({
    "messages": [HumanMessage("Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev")]
})

{'messages': [HumanMessage(content='Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev', additional_kwargs={}, response_metadata={}, id='2a10ac4f-b4d9-4a70-9703-4d958222c7bc'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 359, 'total_tokens': 416, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-f0544918-6d6d-40a1-a4c3-7ad522fdaedf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--a8f91209-be4e-486a-98f8-09bb2495d9b1-0', tool_calls=[{'name': 'save_user_info', 'args': {'user_id': 'abc123', 'user_info': {'name': 'Foo', 'age': 25, 'email': 'foo@langchain.dev'}}, 'id': 'call_02db0e9b643f4eb3812eaf70', 'type': 'tool_call'}], usage_metadata={'input_tokens': 359, 'output_tokens': 57, 'total_tokens': 4

In [10]:
# Second session: get user info
agent.invoke({
    "messages": [HumanMessage("Get user info for user with id 'abc123'")]
})

{'messages': [HumanMessage(content="Get user info for user with id 'abc123'", additional_kwargs={}, response_metadata={}, id='4574a032-3c58-4960-b632-87bed64eba95'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 343, 'total_tokens': 368, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-3af0772d-744c-4038-99ed-6a5e25747962', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--5e82f89c-15fa-4550-b9ce-fb72b5526cc8-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'abc123'}, 'id': 'call_aff8d09a0fab42ea85af77ce', 'type': 'tool_call'}], usage_metadata={'input_tokens': 343, 'output_tokens': 25, 'total_tokens': 368, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  ToolMessage(content="{'name': 'Foo', 'ag

In [11]:
from langchain.tools import tool, ToolRuntime


@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    writer = runtime.stream_writer

    # Stream custom updates as the tool executes
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    return f"It's always sunny in {city}!"